In [80]:
#!/usr/bin/env python

In [81]:
import numpy as np
import sys
from numpy.linalg import eig
from numpy.linalg import svd
from tools import *

In [82]:
np.set_printoptions(precision=4)
np.set_printoptions(threshold=30)
np.set_printoptions(linewidth=300)
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=sys.maxsize)

This code tests Proposition 9 of the paper <br>
ONLEARNING WITHINTEGRALOPERATORS<br>
The idea is that we know what the relationship between eigenvectors of the kernel matrix and the eigen functions.

Given $\psi_i$ as the feature map and $n$ as the number of samples, we define the integral operator $T_n$ as <br>
$$T_n = \frac{1}{n}\sum_i \psi_i(x_i) \psi_i^T(x_i) $$

In this experiment, we will use  the feature map of a polynomial kernel <br>
$k(x,y) = (x^Ty + 1)^2$<br>
We will assume that each sample has two dimensions, therefore the feature map is finite <br>
$\psi(x) = [x_1^2,  \sqrt{2} x_1x_2, \sqrt{2} x_1, \sqrt{2} x_2, x_2^2, 1]$<br>
By using this equation, we can compute Tn, the approximate integral operator. 

In [83]:
def polyK(X):	# calculate the polynomial kernel matrix
	K = np.zeros((len(X), len(X)))
	for i, x in enumerate(X):
		for j, y in enumerate(X):
			xi = x.reshape(len(x), 1)
			yi = y.reshape(len(y), 1)
			K[i,j] = (xi.T.dot(yi) + 1)**2
	return K

In [84]:
def Ψ(X):		# this function takes x and calculate its feature map
	fm = np.empty((6, 0))
	c = np.sqrt(2)
	for x in X:
		# feature map of polynomial [x₁ᒾ,  ᴄ x₁x₂, ᴄ x₁, c x₂, x₂ᒾ, 1]
		ψ = np.array([[x[0]*x[0], c*x[0]*x[1], c*x[0], c*x[1], x[1]*x[1], 1]]).T
		fm = np.hstack((fm, ψ))
	return fm.T

In [85]:
if __name__ == "__main__":
	jprint = jupyter_print

In [86]:
	X = np.array([[1,1], [2,2], [3,3]])
	n = X.shape[0]
	#	
	Ψ = Ψ(X)						# feature map
	K = polyK(X)					# X to kernel matrix
	#
	Tn = (1/n)*Ψ.T.dot(Ψ)			# The approximate integral operator
	#
	[λ, V] = eig(K)				# eigenvalue/vector of the kernel matrix
	[σ, U] = eig(Tn)				# eigenvalue/function of the integral operator

In [87]:
	#
	eK = pretty_np_array(λ, front_tab='', title='Eig Values of K', auto_print=False)
	eQ = pretty_np_array(σ, front_tab='', title='Eig Values of Tn', auto_print=False)
	block_two_string_concatenate(eK, eQ, spacing='\t', add_titles=[], auto_print=True)
	
	#Notice that the eigenvectors of K is not the same as $T_n$!!!
	#However if we multiply the eigenvalues by n, they become the same.
#
	nσ = n*σ[0:3]
	σ_text = pretty_np_array(nσ, front_tab='', title='Eigenvalues after multiplied by n', auto_print=False, end_space=" = 3 * ") 
	eQn = pretty_np_array(σ[0:3], front_tab='', title='Original eigenvalues', auto_print=False)
	display1 = block_two_string_concatenate(σ_text, eQn, spacing=' ', add_titles=['Eigs'], auto_print=False)
	jprint(display1)

      Eig Values of K       	Eig Values of Tn                   
[447.3522   3.6082   0.0397]	[149.1174   1.2027   0.0132  -0.       0.       0.    ]
                            	




Therefore, we conclude the relationship between eigenvalues of kernel matrix and $T_n$ is<br>
$$\mathbf{\sigma(K) = n*\sigma(T_n)}$$

We next verify the eigenvector eigenfunction relationship.<br>
We first show the eigenvectors of K and eigenfunctions from Tn

In [88]:
	eigK = pretty_np_array(V, front_tab='', title='Eig of K', auto_print=False)
	eigQ = pretty_np_array(U, front_tab='', title='Eig of Tn', auto_print=False)
	block_two_string_concatenate(eigK, eigQ, spacing='\t', add_titles=[], auto_print=True)
#

         Eig of K        	Eig of Tn                    
[-0.1245 -0.7546  0.6442]	[-0.4678 -0.1666  0.0582 -0.3224  0.6895 -0.0053]
[-0.4226 -0.5471 -0.7225]	[-0.6616 -0.2356  0.0824  0.6878  0.0243  0.0024]
[-0.8977  0.3622  0.2508]	[-0.2449  0.5676 -0.3433 -0.     -0.     -0.7071]
                         	[-0.2449  0.5676 -0.3433  0.      0.      0.7071]



Given $\phi_i$ as the $i_{th}$ eigenfunction associated with the eigenvector of $K$ denoted as $v_i$.<br>
Given $\sigma_i$ as the eigenvalues of the integral operator (not from the kernel)<br>
Note that the relationship between the eigenvectors of K and eigenfunction is<br>
$$\phi_i = \frac{1}{\sqrt{n \sigma_i}} \Psi^T v_i$$

where<br>
$$\Psi^T = \begin{bmatrix} \psi(x_1) &  \psi(x_2) & ... & \psi(x_n) \end{bmatrix}$$

Notice that our eigenvector eigenfunction relationship is slightly different from the equation proposed by <br>
Rosasco, and that is because he has defined the kernel matrix differently. 

In [89]:
	σ3 = σ[0:3]
	eigFun_from__eigV_of_K = (1/np.sqrt(n*σ3))*((Ψ.T.dot(V)))
	ϕ = U[:,0:3]
#
	jprint('Notice how the eigenfunction computed from the eigenvectors is identical to the actual eigenfunctions !!!')
	eigFun_from__eigV_of_K = pretty_np_array(eigFun_from__eigV_of_K, front_tab='', title='Computed eig Function', auto_print=False)
	eigFun_str = pretty_np_array(ϕ, front_tab='', title='True Eigen Function', auto_print=False)
	block_two_string_concatenate(eigFun_from__eigV_of_K, eigFun_str, spacing='\t', add_titles=[], auto_print=True)

  Computed eig Function  	True Eigen Function   
[-0.4678  0.1666  0.0582]	[-0.4678 -0.1666  0.0582]
[-0.6616  0.2356  0.0824]	[-0.6616 -0.2356  0.0824]
[-0.2449 -0.5676 -0.3433]	[-0.2449  0.5676 -0.3433]
[-0.2449 -0.5676 -0.3433]	[-0.2449  0.5676 -0.3433]
[-0.4678  0.1666  0.0582]	[-0.4678 -0.1666  0.0582]
[-0.0683 -0.4946  0.8664]	[-0.0683  0.4946  0.8664]
                         	




Alternatively, if we let $\lambda_i$ be the eigenvalues from the kernel matrix K, then the relationship would be<br>
$$\phi_i = \frac{1}{\sqrt{\lambda_i}} \Psi^T v_i$$

In [90]:
	eigFun_from__eigV_of_K = (1/np.sqrt(λ))*((Ψ.T.dot(V)))
	ϕ = U[:,0:3]
#
	jprint('Notice how the eigenfunction computed from the eigenvectors is identical to the actual eigenfunctions !!!')
	eigFun_from__eigV_of_K = pretty_np_array(eigFun_from__eigV_of_K, front_tab='', title='Computed eig Function', auto_print=False)
	eigFun_str = pretty_np_array(ϕ, front_tab='', title='True Eigen Function', auto_print=False)
	block_two_string_concatenate(eigFun_from__eigV_of_K, eigFun_str, spacing='\t', add_titles=[], auto_print=True)

  Computed eig Function  	True Eigen Function   
[-0.4678  0.1666  0.0582]	[-0.4678 -0.1666  0.0582]
[-0.6616  0.2356  0.0824]	[-0.6616 -0.2356  0.0824]
[-0.2449 -0.5676 -0.3433]	[-0.2449  0.5676 -0.3433]
[-0.2449 -0.5676 -0.3433]	[-0.2449  0.5676 -0.3433]
[-0.4678  0.1666  0.0582]	[-0.4678 -0.1666  0.0582]
[-0.0683 -0.4946  0.8664]	[-0.0683  0.4946  0.8664]
                         	




Next, we can get the inverse relationship where we go from the eigenfunction back to the eigenvector. <br>
We use the following derivation<br>
$$\phi_i = \frac{1}{\sqrt{\lambda_i}} \Psi^T v_i$$<br>
$$\Psi \phi_i = \frac{1}{\sqrt{\lambda_i}} \Psi \Psi^T v_i$$<br>
$$\Psi \phi_i = \frac{1}{\sqrt{\lambda_i}} K v_i$$<br>
$$\Psi \phi_i = \frac{\lambda_i}{\sqrt{\lambda_i}} v_i$$<br>
$$\Psi \phi_i = \sqrt{\lambda_i} v_i$$<br>
$$\frac{1}{\sqrt{\lambda_i}}\Psi \phi_i = v_i$$<br>
$$\frac{1}{\sqrt{\lambda_i}}\Psi \begin{bmatrix} f_1 & f_2 & .. \end{bmatrix} = \begin{bmatrix} v_1 & v_2 & .. \end{bmatrix}$$

In [91]:
	
	original_u = (1/np.sqrt(λ))*((Ψ.dot(ϕ)))
	computedV = pretty_np_array(original_u, front_tab='', title='Computed eigvector', auto_print=False)
	actualV = pretty_np_array(V, front_tab='', title='True Eigenvector', auto_print=False)
	block_two_string_concatenate(computedV, actualV, spacing='\t', add_titles=[], auto_print=True)

    Computed eigvector   	True Eigenvector    
[-0.1245  0.7546  0.6442]	[-0.1245 -0.7546  0.6442]
[-0.4226  0.5471 -0.7225]	[-0.4226 -0.5471 -0.7225]
[-0.8977 -0.3622  0.2508]	[-0.8977  0.3622  0.2508]
                         	




Lastly, note that you can map the data onto RKHS via the eigenfunctions $\phi_i$ and reproduce the kernel

In [92]:
	L = Ψ.dot(ϕ)
	K2 = L.dot(L.T)

In [93]:
	computedK = pretty_np_array(K2, front_tab='', title='Computed eigvector', auto_print=False)
	actualK = pretty_np_array(K, front_tab='', title='True Eigenvector', auto_print=False)
	block_two_string_concatenate(computedK, actualK, spacing='\t', add_titles=[], auto_print=True)

Computed eigvector	True Eigenvector
[  9.  25.  49]   	[  9.  25.  49]
[ 25.  81. 169]   	[ 25.  81. 169]
[ 49. 169. 361]   	[ 49. 169. 361]
                  	


